In [8]:
import pandas as pd

In [9]:
#import boston311 stuff
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


In [10]:
data = Boston311LogReg(train_date_range={'start':'2010-12-31','end':'2023-12-31'}).load_data()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:263: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:263: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:263: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:263: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:263: DtypeWarning: Columns (13) have mixed types. Specify dtype opt

Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [11]:
#save data to csv as all_311_cases.csv
data.to_csv('all_311_cases.csv', index=False)

In [12]:
#save data to a pkl file as all_311_cases.pkl
data.to_pickle('all_311_cases.pkl')

In [13]:
#load empty_response_ids.csv, remove duplicates, and save it again as empty_response_ids.csv
data = pd.read_csv('empty_response_ids.csv')
data = data.drop_duplicates(subset=['case_enquiry_id'])
data.to_csv('empty_response_ids.csv', index=False)

In [14]:
import pandas as pd
import requests
import csv
import time
import json
import os

# Load your CSVs into pandas DataFrames

case_enquiry_id_file = "all_311_cases.csv"
service_request_id_file = "all_311_cases_api.csv"
case_enquiry_id_pkl = "all_311_cases.pkl"
empty_response_file = "empty_response_ids.csv"

#check if pkl file exists
if os.path.exists(case_enquiry_id_pkl):
    print("pkl file exists")
    #check if pkl file modified date is older than csv file
    if os.path.getmtime(case_enquiry_id_pkl) < os.path.getmtime(case_enquiry_id_file):
        print("pkl file is older than csv file")
        #delete pkl file
        os.remove(case_enquiry_id_pkl)
        print("pkl file deleted")
        #read csv file
        df1 = pd.read_csv(case_enquiry_id_file)
        #save csv file as pkl file
        df1.to_pickle(case_enquiry_id_pkl)
    else:
        print("pkl file is newer than csv file")
        df1 = pd.read_pickle(case_enquiry_id_pkl)
else:
    print("pkl file does not exist")
    df1 = pd.read_csv(case_enquiry_id_file)
    df1.to_pickle(case_enquiry_id_pkl)



#file with service_request_id
df2 = pd.read_csv(service_request_id_file)

#check if empty_response_file exists
if os.path.exists(empty_response_file):
    print("empty_response_file exists")
    df3 = pd.read_csv(empty_response_file)


# Identify missing service_request_ids
missing_ids = set(df1['case_enquiry_id']) - set(df2['service_request_id'])

#also subtract the empty_response_file
if df3 is not None:
    missing_ids = missing_ids - set(df3['case_enquiry_id'])



missing_records = df1[df1['case_enquiry_id'].isin(missing_ids)]
missing_records = missing_records.sort_values(by='case_enquiry_id', ascending=False)

#initialize CSV file for empty response IDs
#if the file exists, append to it
if os.path.exists(empty_response_file):
    empty_response_csv_file = open(empty_response_file, 'a', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
else:
    empty_response_csv_file = open(empty_response_file, 'w', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
    empty_response_csv_writer.writerow(['case_enquiry_id'])

# Initialize CSV file
csv_file_path = service_request_id_file
csv_file = open(csv_file_path, 'a', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Rate limit delay
rate_limit_delay = 6  # 6 seconds to stay within 10 requests per minute
max_exponential_backoff = 3600 

for service_request_id in missing_records['case_enquiry_id']:
    url = f"https://311.boston.gov/open311/v2/requests.json?service_request_id={service_request_id}"

    try:
        response = requests.get(url)
        
        print(f"Fetching data for service_request_id {service_request_id}")

        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                if not data:  # if data is empty
                    empty_response_csv_writer.writerow([service_request_id])
                    empty_response_csv_file.flush()
                    print(f"Empty response for service_request_id {service_request_id}")
                else:
                    for record in data:
                        #print all the data
                        #print(record.get('service_request_id'), record.get('status'), record.get('service_name'), record.get('service_code'), record.get('description'), record.get('requested_datetime'), record.get('updated_datetime'), record.get('address'), record.get('lat'), record.get('long'), record.get('token'))
                        csv_writer.writerow([
                            record.get('service_request_id'),
                            record.get('status'),
                            record.get('service_name'),
                            record.get('service_code'),
                            record.get('description'),
                            record.get('requested_datetime'),
                            record.get('updated_datetime'),
                            record.get('address'),
                            record.get('lat'),
                            record.get('long'),
                            record.get('token')
                        ])
                        csv_file.flush()
                        print(f"Data written for service_request_id {service_request_id}")
                    rate_limit_delay = 6
            except json.JSONDecodeError:
                print(f"Failed to decode JSON for service_request_id {service_request_id}")
                # Double the delay time after failure
                rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
                print(f"Waiting {rate_limit_delay} seconds before retrying")
        else:
            print(f"Failed to fetch data for service_request_id {service_request_id}")
            # Double the delay time after failure
            rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
            print(f"Waiting {rate_limit_delay} seconds before retrying")   
    except requests.exceptions.ConnectionError:
        print(f"Failed to connect to {url}")
        # Double the delay time after failure
        rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
        print(f"Waiting {rate_limit_delay} seconds before retrying")

    # Rate limiting
    time.sleep(rate_limit_delay)

# Close CSV file
csv_file.close()


pkl file exists
pkl file is newer than csv file
empty_response_file exists
Fetching data for service_request_id 101005106692
Empty response for service_request_id 101005106692
Fetching data for service_request_id 101005106690
Empty response for service_request_id 101005106690
Fetching data for service_request_id 101005106688
Empty response for service_request_id 101005106688
Fetching data for service_request_id 101005106686
Data written for service_request_id 101005106686
Fetching data for service_request_id 101005106685
Data written for service_request_id 101005106685
Fetching data for service_request_id 101005106684
Data written for service_request_id 101005106684
Fetching data for service_request_id 101005106683
Data written for service_request_id 101005106683
Fetching data for service_request_id 101005106681
Data written for service_request_id 101005106681
Fetching data for service_request_id 101005106680
Data written for service_request_id 101005106680
Fetching data for service_re

KeyboardInterrupt: 